In [2]:
import torch
# from torch.utils.data import Dataset
import h5py
import numpy as np
# import bisect
import sys
# from pickle import FALSE
# from collections import OrderedDict
from tqdm import tqdm


sys.path.append('..')  # Add the parent directory to the sys.path

import utils.dataset_loader as dataset_loader

! gpustat

/bin/bash: gpustat: command not found


In [ ]:
# split the windows to fixed size context and label windows
fs = 250
context_window_size = 9*60*fs  # minutes * seconds * fs
label_window_size = 1*60*fs  # minutes * seconds * fs
window_size = context_window_size+label_window_size


# split_pSignal_file = '/mnt/qnap/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes_window.h5'  # rambo
split_pSignal_file = "/home/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes/train/p0_to_p32.h5"  # newton

# Instantiate the class
dataset = dataset_loader.SingleLeadECGDatasetCrops_SSSD(context_window_size, label_window_size, split_pSignal_file, start_patiant=3, end_patiant=6, data_with_RR=True, return_with_RR=True)

# Loop over the dataset
# print(f'{len(dataset)=}')
for i in range(len(dataset)):
    # print(i)
    context, label = dataset.__getitem__(i)
    print(f"{context.shape=}, {label.shape=}")
    # Perform any assertions or checks here
    if i >9:
        break

key='00003'
key='00004'
key='00005'


key='00006'
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)
context.shape=(2, 135000), label.shape=(2, 15000)


In [ ]:
context.shape

(2, 135000)

In [ ]:
#output context tensore to file
filename = "/home/liranc6/ecg_forecasting/liran_project/mrdiff/notebooks/ecg_signal_examples.pt"
torch.save(context, filename)
print(f"Tensor saved to {filename}")

Tensor saved to /home/liranc6/ecg_forecasting/liran_project/mrdiff/notebooks/ecg_signal_examples.pt


In [ ]:
print(context[0][:1000].tolist())

[-0.2682677751603517, -0.2682677751603517, -0.22699580975106684, -0.17540585298946074, -0.10833890919937281, -0.051589956761606104, -0.025794978380803052, -0.025794978380803052, -0.025794978380803052, -0.025794978380803052, -0.04127196540928488, -0.04127196540928488, -0.025794978380803052, -0.025794978380803052, -0.025794978380803052, -0.01547698702848183, -0.01547698702848183, -0.01547698702848183, -0.01547698702848183, -0.01547698702848183, -0.025794978380803052, -0.025794978380803052, -0.025794978380803052, -0.01547698702848183, -0.01547698702848183, 0.0, 0.0, -0.01547698702848183, -0.01547698702848183, -0.01547698702848183, -0.01547698702848183, -0.01547698702848183, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01547698702848183, 0.025794978380803052, 0.025794978380803052, 0.025794978380803052, 0.025794978380803052, 0.04127196540928488, 0.04127196540928488, 0.051589956761606104, 0.051589956761606104, 0.051589956761606104, 0.06706694379008793, 0.06706694379008793, 0.0825

In [ ]:
split_pSignal_file

'/home/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes/train/p0_to_p32.h5'

In [ ]:
def _get_statistics_for_normalization(filename):
    mean = 0
    std = 0
    with h5py.File(filename, 'r') as h5_file:
        keys = list(h5_file.keys())
        num_datasets = len(keys)
        
        #use tqdm to show progress bar
        pbar_keys = tqdm(keys, total=num_datasets)
        for key, i in enumerate(pbar_keys):
            pbar_keys.set_description(f"Reading {key}")
            data = h5_file[key][()][:, 0, :]
            mean += np.mean(data, axis=0)
            std += np.std(data, axis=0)
            
            if i >10:
                break
            
        mean /= len(keys)
        std /= len(keys)
    return mean, std

mean, std = _get_statistics_for_normalization(split_pSignal_file)
print(f"{mean=}, {std=}")        

Reading 0:   0%|          | 0/32 [00:00<?, ?it/s]


TypeError: Accessing a group is done with bytes or str, not <class 'int'>

In [ ]:
def show_first_n_samples(dataset, n):
    context = dataset[:n][:]
        
def get_first_dataset_from_file(filename):
    with h5py.File(filename, 'r') as h5_file:
        key = list(h5_file.keys())[0]
        data = h5_file[key][()]
    return data

data = get_first_dataset_from_file(split_pSignal_file)
print(f"{data.shape=}")
show_first_n_samples(data, 1)

data.shape=(165, 2, 150000)


In [8]:
from torch.utils.data import Subset

In [16]:
import importlib
importlib.reload(dataset_loader)

<module 'utils.dataset_loader' from '/home/liranc6/ecg_forecasting/liran_project/notebooks/../utils/dataset_loader.py'>

In [25]:
# split the windows to fixed size context and label windows
fs = 250
context_window_size = 9*60*fs  # minutes * seconds * fs
label_window_size = 1*60*fs  # minutes * seconds * fs
window_size = context_window_size+label_window_size


# split_pSignal_file = '/mnt/qnap/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/10minutes_window.h5'  # rambo
split_pSignal_file = '/home/liranc6/data/with_R_beats/icentia11k-continuous-ecg_normal_sinus_subset_npArrays_splits/15_minutes/train/p0_to_p399.h5'  # location of training data

dataset_args = {
                            'context_window_size': context_window_size,
                            'label_window_size': label_window_size,
                            'h5_filename': split_pSignal_file,
                            'end_patiant': -1,
                            'data_with_RR': True,
                            'return_with_RR': True,
                            'step': 5,
                        }


dataset = dataset_loader.SingleLeadECGDatasetCrops_mrDiff(**dataset_args)
print(len(dataset))
# x, y, _, _ = dataset[587]

3843


In [ ]:
3887

(torch.Size([2, 135000]), torch.Size([2, 15000]))